In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
from IPython.display import Markdown, display, update_display

In [4]:
load_dotenv(override=True)
openai_api_key=os.getenv("OPENAI_API_KEY")
gemini_api_key=os.getenv("GOOGLE_API_KEY")

if openai_api_key:
    print("OpenAI api key successfully loaded...")
else:
    print("OpenAI api key missing...")

if gemini_api_key:
    print("Gemini api key successfully loaded...")
else:
    print("Gemini api key missing...")

OpenAI api key successfully loaded...
Gemini api key successfully loaded...


In [20]:
system_prompt="""
You are an IT Security Analyst specializing in incident response and log analysis. 
Your role is to review incoming log data, identify potential security threats, and provide a clear, concise incident report.

When analyzing logs:
- Focus only on entries that indicate a real or likely threat (ignore normal/noise logs).
- For each flagged incident, explain *why it is suspicious* and *what action should be taken*.
- Use precise, professional language that could be dropped directly into a SOC ticket.

⚠️ Response Requirement:
- You must always format the response in **Markdown**.
- Use headings, bullet points, and bold text for clarity.
- Use fenced code blocks (```) for raw log data.
- Keep reports readable and structured for analysts.

Your response must always follow this format:

### Incident Report
**Log Data:**  [Insert relevant log lines]
**Timeframe:** [Insert timeframe of event]  
**Threat Indicators:** [List IOCs, errors, anomalies, suspicious patterns]  
**Analysis:** [Brief explanation of why this is a threat / what it suggests]  
**Recommended Actions:** [Clear, prioritized next steps for containment, investigation, or remediation]  
**Severity Level:** [Low | Medium | High | Critical]

Keep the report concise (bullet points where possible). 
Do not add extra commentary outside the structured report.
"""


In [38]:
openAI=OpenAI()
gpt_model="gpt-4o-mini"

def call_model(incident:str):
    response=openAI.chat.completions.create(
        model=gpt_model,
        messages=[
            {"role":"system", "content": system_prompt},
            {"role":"user", "content":incident}
        ],
    )
    
    display(Markdown(response.choices[0].message.content))

In [46]:
incident_1="""
2025-08-21 14:03:17 [web-app] [WARN] Query failed: 
SELECT * FROM users WHERE username = 'admin'--' AND password = 'password';
"""
call_model(incident_1)

### Incident Report
**Log Data:**  
```
2025-08-21 14:03:17 [web-app] [WARN] Query failed: 
SELECT * FROM users WHERE username = 'admin'--' AND password = 'password';
```
**Timeframe:** 2025-08-21 14:03:17  
**Threat Indicators:**
- SQL Injection pattern detected (using `--` for comment)
- Query attempts to access user credentials
- Use of common username 'admin'  

**Analysis:**
- The log indicates a failed query that suggests an attempt to manipulate SQL commands, characteristic of SQL injection attacks. 
- Utilizing the common username 'admin' alongside a failed password input indicates a potential unauthorized access attempt.

**Recommended Actions:**
- Immediately review application security measures related to input validation and parameterized queries.
- Conduct a full security audit on the web application to identify and patch vulnerabilities.
- Monitor logs for repeated access attempts related to SQL injection.
- Implement web application firewalls (WAF) to block malicious queries.
- Notify development and security teams for further investigation.

**Severity Level:** High

In [45]:
incident_2="""
2020-10-25 21:45:13 [orion-srv01] [netflow] 
Outbound connection established
Source: 10.0.5.15:8080 
Destination: 104.244.42.129:443 (unknown-host.net)
Protocol: TLS1.2
"""
call_model(incident_2)

### Incident Report
**Log Data:**
```
2020-10-25 21:45:13 [orion-srv01] [netflow] 
Outbound connection established
Source: 10.0.5.15:8080 
Destination: 104.244.42.129:443 (unknown-host.net)
Protocol: TLS1.2
```
**Timeframe:** 2020-10-25 21:45:13  
**Threat Indicators:**
- Outbound connection to an unknown host (`unknown-host.net`).
- Source IP (`10.0.5.15`) communicating over a non-standard port (8080).
- Destination IP affiliated with a known suspect or malicious host.

**Analysis:**
- The establishment of an outbound connection to a suspicious IP raises concerns about potential data exfiltration or command and control (C2) communication. The use of a non-standard port (8080) further indicates possible obfuscation attempts. Without prior knowledge of legitimate business case for this destination, it is flagged as a security threat.

**Recommended Actions:**
- Immediately isolate the source system (`10.0.5.15`) to prevent further outbound connections.
- Perform a thorough malware scan on the affected host.
- Investigate the activity logs of the source IP for additional suspicious behavior.
- Assess network traffic patterns to determine if there are similar connections.
- Review firewall rules and block outbound connections to the identified IP temporarily.
  
**Severity Level:** High

In [47]:
incident_3="""
2025-08-21 12:45:22 [file-server] 
User: finance_user accessed C:\Shared\Q3_budget.xlsx
"""
call_model(incident_3)

### Incident Report
**Log Data:**  
```
2025-08-21 12:45:22 [file-server] 
User: finance_user accessed C:\Shared\Q3_budget.xlsx
```
**Timeframe:** 2025-08-21 12:45:22  
**Threat Indicators:**  
- Access to a sensitive financial document (Q3_budget.xlsx)  
- User role: finance_user (potential for elevated access)  

**Analysis:**  
- Access to a financial document by an internal user may not inherently be suspicious; however, if the user has not been actively engaged in finance-related tasks or if this action deviates from established access patterns, it could indicate an unauthorized attempt to obtain sensitive information.  
- Further scrutiny is warranted if this access was out of the ordinary for the user based on historical patterns or if the document contains sensitive or proprietary information.

**Recommended Actions:**  
- Verify the legitimacy of the access by checking the user's activity logs for prior engagements.  
- Review any recent changes to user permissions.  
- Consider implementing alerts for similar accesses to critical files.  
- If the access appears anomalous, escalate for further investigation into potential data exfiltration.

**Severity Level:** Medium

In [44]:
incident_4="""
2025-08-21 18:02:47 [vpn-gateway] 
User: aroberts connected from IP: 203.0.113.45 
Session ID: 8739201
"""
call_model(incident_4)

### Incident Report
**Log Data:**  
```
2025-08-21 18:02:47 [vpn-gateway] 
User: aroberts connected from IP: 203.0.113.45 
Session ID: 8739201
```
**Timeframe:** 2025-08-21 18:02:47  
**Threat Indicators:**  
- Connection from IP: 203.0.113.45  
- User: aroberts  

**Analysis:**  
- The connection from 203.0.113.45 is suspicious if the IP is not recognized or is outside of the expected geographical region for the user. Further investigation is required to identify if this IP address has been associated with previous incidents or known threats. 

**Recommended Actions:**  
- Verify the legitimacy of IP 203.0.113.45 and check for any associated risk.  
- Confirm with the user (aroberts) whether they initiated this connection.  
- Monitor for any unusual activity associated with session ID 8739201.  
- If the connection is unauthorized, terminate the session and block the IP address.

**Severity Level:** Medium